# Test and visualize Deeplomatics Dataset

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import albumentations as A 
from datasets import Deeplomatics
from datasets import draw_rect
from torchvision import models
from my_utils import collate_fn
import torchvision.transforms.functional as TF
from my_models import eval_mAP,eval_accuracy,MetricBuilder
import torch


In [ ]:
gpu = 3
device  = torch.device("cuda:"+str(gpu) if torch.cuda.is_available() else "cpu")

In [ ]:
DATASET_DIR = '/share/DEEPLEARNING/datasets/deeplomatics'
LOAD_DIR = '/share/homes/karmimy/deeplomatics/save_model/image_deeplo'
dir = '11'
model_name = 'deeplo_image.pt'
LOAD_PT = os.path.join(LOAD_DIR,dir,model_name)
model = torch.load(LOAD_PT,map_location=device)
print('')

In [ ]:
dataset = Deeplomatics(DATASET_DIR,image_set='test',normalize=True)
dataloader_test = torch.utils.data.DataLoader(dataset, batch_size=4,shuffle=True,drop_last=True,collate_fn=collate_fn)

In [ ]:
img,targets = dataset.__getitem__(0)
true_bbox,true_label = targets['boxes'],targets['labels']
x = torch.unsqueeze(img,0).to(device)
x = TF.resize(x,(512,512)) 
d = model(x)

In [ ]:
d[0].size()

In [ ]:
x.size()

In [ ]:
DATASET_DIR = '/share/DEEPLEARNING/datasets/deeplomatics'
LOAD_DIR = '/share/homes/karmimy/deeplomatics/save_model/image_deeplo'
dir = '9'
model_name = 'deeplo_image.pt'
LOAD_PT = os.path.join(LOAD_DIR,dir,model_name)
device  = torch.device("cuda:"+str(gpu) if torch.cuda.is_available() else "cpu")

In [ ]:
device  = torch.device("cuda:"+str(gpu) if torch.cuda.is_available() else "cpu")
model = torch.load(LOAD_PT,map_location=device)

In [ ]:
NUMBER_OF_EXAMPLE = 5
PRINT = True
LIMIT = False
n_data = len(dataset)
ind = np.arange(n_data)
i_ex = np.random.choice(ind, size=NUMBER_OF_EXAMPLE, replace=False)

In [ ]:
for i in i_ex : 
    fig = plt.figure()
    model.eval()
    img,targets = dataset.__getitem__(i)
    true_bbox,true_label = targets['boxes'],targets['labels']
    x = torch.unsqueeze(img,0).to(device)
    d = model(x)
    bbox = d[0]['boxes']
    label = d[0]['labels']
    plt.title('Image'+str(i))
    plt.xlabel('True label'+str(true_label)+'; Label pred:'+str(label))
    print('Label pred',label)
    print('True label',true_label)
    img = img.transpose_(0,2).transpose_(0,1)

    binks = draw_rect(img.detach().cpu().numpy(),bbox.detach().cpu().numpy())
    plt.imshow(binks)
    plt.savefig('./results/'+str(i)+'.jpg')

In [ ]:
d

In [ ]:
model.eval()
metric_fn = MetricBuilder.build_evaluation_metric("map_2d", async_mode=False, num_classes=2)
for i,(images,targets) in enumerate(dataloader_test):
    
    if i > NUMBER_OF_EXAMPLE and LIMIT:
        break
    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) for k,v in t.items()} for t in targets] 
    pred_dict = model(images)
    if PRINT : 
        print('----------------------------------------------------------------')
        print('PRED DICT BOXES:',pred_dict[0]['boxes'],'|||| LABEL:',pred_dict[0]['labels'],'...')
        print('****************************************************************')
        print('GROUND TRUTH:',targets[0]['boxes'],'|||| LABEL:',targets[0]['labels'],'...')
        print('----------------------------------------------------------------')
        print('\n\n')
        metric_fn= convert_for_eval(metric_fn,pred_dict,targets) # also add predictions for the metric in this function 

In [ ]:
# compute PASCAL VOC metric
print(f"VOC PASCAL mAP: {metric_fn.value(iou_thresholds=0.5, recall_thresholds=np.arange(0., 1.1, 0.1))['mAP']}")

# compute PASCAL VOC metric at the all points
print(f"VOC PASCAL mAP in all points: {metric_fn.value(iou_thresholds=0.5)['mAP']}")

# compute metric COCO metric
print(f"COCO mAP: {metric_fn.value(iou_thresholds=np.arange(0.5, 1.0, 0.05), recall_thresholds=np.arange(0., 1.01, 0.01), mpolicy='soft')['mAP']}")

In [ ]:

img,target = dataset.__getitem__(12)
pred_dict = model(img.unsqueeze(0).to(device))


In [ ]:
img.size()
plt.imshow(img.transpose_(0,2).transpose_(0,1))

In [ ]:
boxes = pred_dict[0]['boxes']
label = pred_dict[0]['labels']
print('LABEL',label)
binks = draw_rect(img.detach().cpu().numpy(),boxes)
plt.imshow(binks)